# Generation of data

In [1]:
# Specifying various imports
import osmnx as ox
import networkx as nx

## Generation of data for Minimising Pickup Time
The following data corresponds to the current positions of drivers and riders.

We generated time taken to travel between driver and rider.
We also generated distance between driver and rider along the road.

### The following code is of finding the shortest distance over road between two points on the map
Here we just give 2 values to the function and then findout the shortest distance between them.

In [ ]:
import osmnx as ox
import networkx as nx
from typing import Tuple, Dict, Union, List
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_distance_and_eta_osm(
    start_coords: Tuple[float, float],
    end_coords: Tuple[float, float],
    speed_kph: float = 50
) -> Dict[str, Union[float, List[int]]]:
    """
    Calculate road distance and estimated time of arrival (ETA) between two points using OSM.
    
    Parameters:
        start_coords (tuple): Latitude and longitude of the starting point (lat, lon).
        end_coords (tuple): Latitude and longitude of the destination (lat, lon).
        speed_kph (float): Average speed in kilometers per hour (default is 50 kph).
        
    Returns:
        dict: Contains distance (in km), duration (in minutes), and the path as a list of nodes.
        
    Raises:
        ValueError: If coordinates are invalid or if no path is found.
        NetworkXNoPath: If no path exists between the points.
    """
    try:
        # Validate coordinates
        for coords in [start_coords, end_coords]:
            if not (-90 <= coords[0] <= 90) or not (-180 <= coords[1] <= 180):
                raise ValueError(f"Invalid coordinates: {coords}")

        # Calculate the bounding box that encompasses both points with some padding
        min_lat = min(start_coords[0], end_coords[0]) - 0.02
        max_lat = max(start_coords[0], end_coords[0]) + 0.02
        min_lon = min(start_coords[1], end_coords[1]) - 0.02
        max_lon = max(start_coords[1], end_coords[1]) + 0.02

        logger.info("Fetching road network from OpenStreetMap...")
        
        # Create a road network graph using the calculated bbox
        G = ox.graph_from_bbox(
            north=max_lat,
            south=min_lat,
            east=max_lon,
            west=min_lon,
            network_type="drive",
            simplify=True
        )
        
        logger.info("Finding nearest nodes...")
        
        # Find the nearest nodes to the start and end points
        orig_node = ox.distance.nearest_nodes(G, start_coords[1], start_coords[0])
        dest_node = ox.distance.nearest_nodes(G, end_coords[1], end_coords[0])
        
        logger.info("Calculating shortest path...")
        
        try:
            # Calculate the shortest path by distance
            shortest_path = nx.shortest_path(G, orig_node, dest_node, weight="length")
        except nx.NetworkXNoPath:
            raise ValueError("No path found between the given points")
        
        # Calculate the total distance of the path
        distance_m = sum(ox.utils_graph.get_route_edge_attributes(G, shortest_path, "length"))
        distance_km = round(distance_m / 1000, 2)  # Convert meters to kilometers and round to 2 decimal places
        
        # Calculate travel time (in minutes) based on average speed
        travel_time_min = round((distance_km / speed_kph) * 60, 1)
        
        return {
            "distance_km": distance_km,
            "duration_min": travel_time_min,
            "path": shortest_path
        }
        
    except Exception as e:
        logger.error(f"Error calculating route: {str(e)}")
        raise

def visualize_route(
    G: nx.Graph,
    path: List[int],
    start_coords: Tuple[float, float],
    end_coords: Tuple[float, float]
) -> None:
    """
    Visualize the route on a map.
    
    Parameters:
        G (nx.Graph): The road network graph
        path (List[int]): List of node IDs representing the path
        start_coords (tuple): Starting coordinates (lat, lon)
        end_coords (tuple): Ending coordinates (lat, lon)
    """
    try:
        # Plot the route
        fig, ax = ox.plot_graph_route(
            G, 
            path,
            route_color='r',
            route_linewidth=2,
            node_size=0,
            bgcolor='white'
        )
        return fig, ax
    except Exception as e:
        logger.error(f"Error visualizing route: {str(e)}")
        raise

# def main():
#     """Example usage of the routing functions."""
#     try:
#         #19.1342, Longitude: 72.9126
#         #Latitude: 19.1324, Longitude: 72.9179
#         # Example coordinates (Mumbai)
#         # start = (19.0760, 72.8777)  # Mumbai CST
#         # end = (19.2183, 72.9781)    # Thane
#         start = (19.1342, 72.9126)
#         end = (19.1324, 72.9179)
        
#         # Calculate route
#         result = get_distance_and_eta_osm(start, end, speed_kph=40)
        
#         print(f"\nRoute Summary:")
#         print(f"Distance: {result['distance_km']:.2f} km")
#         print(f"Estimated Time: {result['duration_min']:.1f} minutes")
        
#         # Create visualization (optional)
#         # Note: This requires additional memory and processing time
#         # bbox = (min(72.8777, 72.9781), 19.0760, max(72.8777, 72.9781), 19.2183)
#         # G = ox.graph_from_bbox(*bbox, network_type="drive")
#         # visualize_route(G, result['path'], start, end)
        
#     except Exception as e:
#         logger.error(f"Error in main: {str(e)}")
#         return 1
#     return 0

# if __name__ == "__main__":
#     main()

INFO:__main__:Fetching road network from OpenStreetMap...
C:\Users\inamp\AppData\Local\Temp\ipykernel_11516\2729809688.py:45: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_bbox(
C:\Users\inamp\AppData\Local\Temp\ipykernel_11516\2729809688.py:45: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = ox.graph_from_bbox(
INFO:__main__:Finding nearest nodes...
INFO:__main__:Calculating shortest path...



Route Summary:
Distance: 1.26 km
Estimated Time: 1.9 minutes


C:\Users\inamp\AppData\Local\Temp\ipykernel_11516\2729809688.py:69: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance_m = sum(ox.utils_graph.get_route_edge_attributes(G, shortest_path, "length"))


In [6]:
import requests

def get_osrm_route(start_coords, end_coords):
    """
    Fetches distance and travel time using OSRM routing service.

    Parameters:
        start_coords (tuple): (latitude, longitude) of the start point.
        end_coords (tuple): (latitude, longitude) of the end point.

    Returns:
        dict: Distance in kilometers, travel time in minutes.
    """
    # OSRM public API endpoint
    base_url = "https://router.project-osrm.org/route/v1/driving"
    
    # Format coordinates for the API
    coordinates = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
    
    # API request
    response = requests.get(f"{base_url}/{coordinates}", params={"overview": "false"})
    data = response.json()
    
    if response.status_code == 200 and data.get("routes"):
        # Extract distance (meters) and duration (seconds)
        distance_m = data["routes"][0]["distance"]
        duration_sec = data["routes"][0]["duration"]
        
        # Convert to desired units
        distance_km = distance_m / 1000  # meters to kilometers
        duration_min = duration_sec / 60  # seconds to minutes
        
        return {
            "distance_km": distance_km,
            "travel_time_minutes": duration_min
        }
    else:
        raise ValueError(f"Error fetching route: {data.get('message', 'Unknown error')}")

# Example usage
if __name__ == "__main__":
    start = (19.1342, 72.9126)  # Hostel-15, IIT Bombay
    end = (19.1324, 72.9179)    # IIT Bombay Main Gate
    
    try:
        route_info = get_osrm_route(start, end)
        print(f"Distance: {route_info['distance_km']:.2f} km")
        print(f"Travel Time: {route_info['travel_time_minutes']:.2f} minutes")
    except ValueError as e:
        print(e)


Distance: 1.12 km
Travel Time: 4.27 minutes
